In [37]:
!pip3 install pydantic openai datasets trl transformers peft torch pandas wandb vllm tqdm fundus arxiv datasets clize xformers flash-attn llama-index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [38]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime, timedelta
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from openai import OpenAI
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
import pprint
import json
import random
from fundus import PublisherCollection, Crawler
from transformers import AutoTokenizer
from llama_index.readers.papers import ArxivReader
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=2.0, min_p=0.3)
#"/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411"
llm = LLM(model="/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411", tensor_parallel_size=4, guided_decoding_backend = "lm-format-enforcer")

conversation = [

    {

        "role": "system",

        "content": "You are a helpful assistant"

    },

    {

        "role": "user",

        "content": "Hello"

    },

    {

        "role": "assistant",

        "content": "Hello! How can I assist you today?"

    },

    {

        "role": "user",

        "content": "Write an essay about the importance of higher education.",

    },

]

outputs = llm.chat(conversation, sampling_params=sampling_params)

for output in outputs:
    print(output.outputs[0].text)

INFO 12-25 05:13:37 config.py:2167] Downcasting torch.float32 to torch.float16.
INFO 12-25 05:13:37 config.py:478] This model supports multiple tasks: {'score', 'generate', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 12-25 05:13:37 config.py:1216] Defaulting to use mp for distributed inference
WARNING 12-25 05:13:37 arg_utils.py:1086] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-25 05:13:37 config.py:1364] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 12-25 05:13:37 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411', speculative_config=None, tokenizer='/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411', skip_t

Loading safetensors checkpoint shards:   0% Completed | 0/51 [00:00<?, ?it/s]


(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc_worker_utils.py:236] Exception in worker VllmWorkerProcess while processing method load_model.
(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc_worker_utils.py:236] Traceback (most recent call last):
(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc_worker_utils.py:236]   File "/home/lain/micromamba/envs/vllm/lib/python3.11/site-packages/vllm/executor/multiproc_worker_utils.py", line 230, in _run_worker_process
(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc_worker_utils.py:236]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc_worker_utils.py:236]              ^^^^^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc_worker_utils.py:236]   File "/home/lain/micromamba/envs/vllm/lib/python3.11/site-packages/vllm/worker/worker.py", line 155, in load_model
(VllmWorkerProcess pid=42485) ERROR 12-25 05:13:45 multiproc

KeyboardInterrupt: 

In [5]:
sampling_params = SamplingParams(temperature=2.0, min_p=0.3, min_tokens=200, max_tokens=500, top_p=1, top_k=-1)

outputs = llm.chat(conversation, sampling_params=sampling_params)

for output in outputs:
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:25<00:00, 25.84s/it, est. speed input: 1.32 toks/s, output: 19.35 toks/s]

 ### The Importance of Higher Education

In an increasingly interconnected and competitive world, the significance of higher education cannot be overstated. Higher education serves as a catalyst for personal growth, intellectual development, and economic prosperity. It equips individuals with the knowledge, skills, and credentials necessary to navigate the complexities of modern society and contribute meaningfully to their communities and the global economy.

#### Personal Growth and Intellectual Development

Higher education fosters personal growth by providing a structured environment for individuals to explore their interests, develop critical thinking skills, and gain a deeper understanding of the world. Universities and colleges offer a diverse range of courses and extracurricular activities that encourage students to step out of their comfort zones and engage with new ideas and perspectives. This exposure broadens their horizons and cultivates a sense of curiosity and lifelong le

INFO 12-24 21:06:39 multiproc_worker_utils.py:140] Terminating local vLLM worker processes
Successfully delete the llm pipeline and free the GPU memory.


(VllmWorkerProcess pid=38872) (VllmWorkerProcess pid=38874) (VllmWorkerProcess pid=38873) INFO 12-24 21:06:39 multiproc_worker_utils.py:247] Worker exiting
INFO 12-24 21:06:39 multiproc_worker_utils.py:247] Worker exiting
INFO 12-24 21:06:39 multiproc_worker_utils.py:247] Worker exiting


In [10]:

'''
client = OpenAI(
    base_url="http://20.81.188.27:8000/v1",
    api_key="token-abc123",
)
'''

def load_arxiv_papers(search_query: str, max_results: int = 1):    
    loader = ArxivReader()
    documents = loader.load_data(search_query=search_query, max_results=max_results)
    return documents

MODEL_NAME = "mistralai/Mistral-Large-Instruct-2411"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

documents = load_arxiv_papers("(multimodality OR multitask learning) AND ((low-resource languages) OR (code-switching)) -SoupLM", max_results=1)

for document in documents:
    print(document.text)


M3P: Learning Universal Representations via Multitask Multilingual
Multimodal Pre-training
Minheng Ni1*† Haoyang Huang2† Lin Su3† Edward Cui3 Taroon Bharti3 Lijuan Wang4
Jianfeng Gao5 Dongdong Zhang2 Nan Duan2‡
1 Research Center for Social Computing and Information Retrieval
Harbin Institute of Technology, China
2 Natural Language Computing, Microsoft Research Asia, China
3 Bing Multimedia Team, Microsoft, China
4 Cloud+AI, Microsoft, United States
5 Deep Learning, Microsoft Research Redmond, United States
mhni@ir.hit.edu.cn
{haohua, lins, edwac, tbharti, lijuanw, jfgao, Dongdong.Zhang, nanduan}@microsoft.com
Abstract
We present M3P, a Multitask Multilingual Multimodal
Pre-trained model that combines multilingual pre-training
and multimodal pre-training into a uniﬁed framework via
multitask pre-training. Our goal is to learn universal repre-
sentations that can map objects occurred in different modal-
ities or texts expressed in different languages into a com-
mon semantic space. In ad

In [81]:
document

Document(id_='a44d6c99-896d-4657-8b84-5d9016aa665f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='The following is a summary of the paper: M3P: Learning Universal Representations via Multitask Multilingual Multimodal Pre-training\n\nSummary: We present M3P, a Multitask Multilingual Multimodal Pre-trained model that\ncombines multilingual pre-training and multimodal pre-training into a unified\nframework via multitask pre-training. Our goal is to learn universal\nrepresentations that can map objects occurred in different modalities or texts\nexpressed in different languages into a common semantic space. In addition, to\nexplicitly encourage fine-grained alignment between images and non-English\nlanguages, we also propose Multimodal Code-switched Training (MCT) to combine\nmonolingual pre-training and 

In [19]:




"""
qa_response = generate_qa(document.text)
my_response = ""
for response in qa_response:
    my_response += response.outputs[0].text

my_response
"""

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.21s/it, est. speed input: 25.83 toks/s, output: 18.80 toks/s]


[RequestOutput(request_id=2, prompt='<s>[SYSTEM_PROMPT] You are an expert at creating high-quality training data for language models.\nGiven academic research content, extract the key concepts and convert them into general knowledge questions and answers.\nQuestions should:\n- Be general and broadly applicable\n- Focus on core concepts and ideas\n- Be written as standalone questions without referencing any specific research\n- Test understanding of the topic area\n\nAnswers should:\n- Provide clear, factual information\n- Be written as standalone knowledge\n- Focus on general concepts rather than specific research\n- Be useful for general learning about the topic[/SYSTEM_PROMPT][INST] Based on these concepts, generate 3 general knowledge questions and answers about: The following is a summary of the paper: M3P: Learning Universal Representations via Multitask Multilingual Multimodal Pre-training\n\nSummary: We present M3P, a Multitask Multilingual Multimodal Pre-trained model that\ncom

In [30]:

"""
evaluation_response = evaluate_qa(document.text, QA.parse_raw(my_response))
evaluation_response

my_eval_response = ""
for response in evaluation_response:
    my_eval_response += response.outputs[0].text

my_eval_response
"""

Processed prompts: 100%|██████████| 1/1 [00:34<00:00, 34.13s/it, est. speed input: 20.92 toks/s, output: 18.87 toks/s]


' \t\t\t\t\t\t\t\t\t\t\t{\n\t"qa_pair1_arguments_for":\n\t[\n\t"The question directly addresses the core concept of multimodal pre-training, which is a central theme in the paper."\n\t,"The answer accurately and clearly summarizes the primary goal of multimodal pre-training, making it a valuable addition to the dataset."\n\t,"The pair emphasizes the importance of learning universal representations, which is a key contribution of the M3P model."\n\t]\n\t,"qa_pair1_arguments_against":\n\t[\n\t"The answer is quite general and might not capture the specific nuances of the M3P model\'s approach to multimodal pre-training."\n\t,"The phrase \'such as images and text\' is not explicitly mentioned in the original text and might introduce slight inaccuracies."\n\t]\n\t,"qa_pair1_include": "y"\n\t,"qa_pair2_arguments_for":\n\t[\n\t"The question focuses on another crucial aspect of the M3P model—multilingual pre-training."\n\t,"The answer accurately explains the purpose of multilingual pre-trainin

In [1]:
import pandas as pd
from pathlib import Path
import json
from tqdm import tqdm
import gc
import contextlib
import ray
import torch
from vllm import LLM, SamplingParams
from vllm.distributed.parallel_state import (
    destroy_model_parallel,
    destroy_distributed_environment,
)
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime, timedelta
from lmformatenforcer import JsonSchemaParser
from openai import OpenAI
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
import pprint
import random
from fundus import PublisherCollection, Crawler
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM
from llama_index.readers.papers import ArxivReader
from datasets import Dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, PeftModel
import wandb
import os
from enum import Enum
from pydantic import BaseModel
from vllm import LLM, SamplingParams
from vllm.sampling_params import GuidedDecodingParams

def load_arxiv_papers(search_query: str, max_results: int = 1):    
    loader = ArxivReader()
    documents = loader.load_data(search_query=search_query, max_results=max_results)
    return documents

class QA(BaseModel):
    question1: str = Field(description="First general knowledge question about the core topic and concepts")
    answer1: str = Field(description="Clear, factual answer focused on general knowledge")
    question2: str = Field(description="Second general knowledge question about the core topic and concepts") 
    answer2: str = Field(description="Clear, factual answer focused on general knowledge")
    question3: str = Field(description="Third general knowledge question about the core topic and concepts")
    answer3: str = Field(description="Clear, factual answer focused on general knowledge")

DEFAULT_SYSTEM_CONTENT = """You are an expert at creating high-quality training data for language models.
Given academic research content, extract the key concepts and convert them into general knowledge questions and answers.
Questions should:
- Be general and broadly applicable
- Focus on core concepts and ideas
- Be written as standalone questions without referencing any specific research
- Test understanding of the topic area

Answers should:
- Provide clear, factual information
- Be written as standalone knowledge
- Focus on general concepts rather than specific research
- Be useful for general learning about the topic"""

def generate_qa(document_text: str, system_content: str = DEFAULT_SYSTEM_CONTENT, feedback: str = None):
    json_schema = QA.model_json_schema()
    guided_decoding_params = GuidedDecodingParams(json=json_schema)
    sampling_params = SamplingParams(
        guided_decoding=guided_decoding_params,
        temperature=1.5,
        max_tokens=4000,
        presence_penalty=0.1,
        frequency_penalty=0.1,
        min_p=0.4,
        stop=["<|eot_id|>"]
    )
    
    messages = [
        {"role": "system", "content": system_content}
    ]
    
    if feedback:
        messages.append({"role": "user", "content": f"Based on this feedback, please generate improved questions and answers:\n\n{feedback}\n\nOriginal text: {document_text}"})
    else:
        messages.append({"role": "user", "content": f"Based on these concepts, generate 3 general knowledge questions and answers about: {document_text}"})
    
    return llm.chat(
        messages=messages,
        sampling_params=sampling_params
    )

class QAEvaluation(BaseModel):
    qa_pair1_arguments_for: list[str] = Field(description="Arguments in favor of including first QA pair")
    qa_pair1_arguments_against: list[str] = Field(description="Arguments against including first QA pair") 
    qa_pair1_include: Literal["y", "n"] = Field(description="Whether to include first QA pair")

    qa_pair2_arguments_for: list[str] = Field(description="Arguments in favor of including second QA pair")
    qa_pair2_arguments_against: list[str] = Field(description="Arguments against including second QA pair")
    qa_pair2_include: Literal["y", "n"] = Field(description="Whether to include second QA pair")

    qa_pair3_arguments_for: list[str] = Field(description="Arguments in favor of including third QA pair")
    qa_pair3_arguments_against: list[str] = Field(description="Arguments against including third QA pair")
    qa_pair3_include: Literal["y", "n"] = Field(description="Whether to include third QA pair")
    
    next_search_query: str = Field(description="Recommended next search query for finding related but diverse papers")

def evaluate_qa(document_text: str, qa_output: QA, round_num: int):
    json_schema = QAEvaluation.model_json_schema()
    guided_decoding_params = GuidedDecodingParams(json=json_schema)
    sampling_params = SamplingParams(
        guided_decoding=guided_decoding_params,
        temperature=1.5,
        max_tokens=4000,
        presence_penalty=0.1,
        frequency_penalty=0.1,
        min_p=0.4,
        stop=["<|eot_id|>"]
    )
    
    eval_system_prompt = f"""You are an expert evaluator of training data quality for language models.
    This is debate round {round_num}. Be increasingly critical with each round.
    For each question-answer pair:
    1. List key arguments for and against including it (considering accuracy, clarity, generalizability)
    2. Make a binary decision (y/n) if it should be included in the final dataset
    
    Additionally, recommend a search query for finding the next paper to analyze. The query should:
    - Be related to but very distinct from the current paper's topic
    - Help build a diverse dataset while maintaining topical coherence
    - Potentially focus on an interesting direction suggested by the current paper"""
    
    eval_user_prompt = f"""Evaluate these QA pairs generated from the paper:
    
    Original Paper Text: {document_text}
    
    QA Pair 1:
    Q: {qa_output.question1}
    A: {qa_output.answer1}
    
    QA Pair 2:
    Q: {qa_output.question2}
    A: {qa_output.answer2}
    
    QA Pair 3:
    Q: {qa_output.question3}
    A: {qa_output.answer3}
    
    Provide a structured evaluation following the schema, including a recommended next search query."""

    messages = [
        {"role": "system", "content": eval_system_prompt},
        {"role": "user", "content": eval_user_prompt}
    ]
    
    return llm.chat(
        messages=messages,
        sampling_params=sampling_params
    )

def archive_enchanted_dialogues(knowledge_exchange_data, assessment_data, archive_path="wand_university_training_grimoire.csv"):
    """Archive validated magical dialogues and their scholarly assessments for the Wand University curriculum"""
    print("\n🌟 INITIATING ARCHIVAL PROCESS OF ENCHANTED DIALOGUES 🌟")
    
    # Transform assessment data from arcane notation to comprehensible format
    assessment_grimoire = json.loads(assessment_data)
    
    # Initialize collection of validated knowledge exchanges
    validated_exchanges = []
    
    # Evaluate each knowledge exchange through our rigorous magical standards
    for exchange_id in tqdm(range(1,4), desc="VALIDATING KNOWLEDGE EXCHANGES"):
        if assessment_grimoire[f'qa_pair{exchange_id}_include'] == 'y':
            # Synthesize supporting and challenging arguments
            supporting_thesis = assessment_grimoire.get(f'qa_pair{exchange_id}_arguments_for', [])
            counterpoints = assessment_grimoire.get(f'qa_pair{exchange_id}_arguments_against', [])
            
            print(f"\n✨ ACCEPTED KNOWLEDGE EXCHANGE {exchange_id}:")
            print(f"📝 INQUIRY: {getattr(knowledge_exchange_data, f'question{exchange_id}')}")
            print(f"💭 RESPONSE: {getattr(knowledge_exchange_data, f'answer{exchange_id}')}")
            print("✅ POSITIVE FEEDBACK:")
            for strength in supporting_thesis:
                print(f"  • {strength}")
            
            validated_exchanges.append({
                'inquiry': getattr(knowledge_exchange_data, f'question{exchange_id}'),
                'wisdom': getattr(knowledge_exchange_data, f'answer{exchange_id}'),
                'supporting_thesis': ', '.join(supporting_thesis),
                'counterpoints': ', '.join(counterpoints),
                'search_query': assessment_grimoire['next_search_query']
            })
        else:
            print(f"\n⚠️ DISCARDED KNOWLEDGE EXCHANGE {exchange_id}:")
            print(f"📝 INQUIRY: {getattr(knowledge_exchange_data, f'question{exchange_id}')}")
            print(f"💭 RESPONSE: {getattr(knowledge_exchange_data, f'answer{exchange_id}')}")
            print("❌ CRITICAL FEEDBACK:")
            for critique in assessment_grimoire.get(f'qa_pair{exchange_id}_arguments_against', []):
                print(f"  • {critique}")
    
    print("\n📚 TRANSFORMING KNOWLEDGE INTO STRUCTURED FORMAT...")
    # Transform into structured knowledge format
    knowledge_codex = pd.DataFrame(validated_exchanges)
    
    # Preserve in the grand archives
    print("💾 PRESERVING IN THE GRAND ARCHIVES...")
    if Path(archive_path).exists():
        knowledge_codex.to_csv(archive_path, mode='a', header=False, index=False, sep='|')
    else:
        knowledge_codex.to_csv(archive_path, index=False, sep='|')
    
    print(f"✨ SUCCESSFULLY ARCHIVED {len(validated_exchanges)} EXCHANGES ✨")
    return len(validated_exchanges)

def synthesize_and_evaluate_knowledge(source_manuscript, research_focus, debate_rounds=3):
    """Synthesize and evaluate magical knowledge exchanges through our proprietary arcane processes"""
    print("\n🔮 INITIATING KNOWLEDGE SYNTHESIS AND EVALUATION 🔮")
    
    # Initial generation
    print("📖 GENERATING INITIAL KNOWLEDGE EXCHANGES...")
    knowledge_response = generate_qa(source_manuscript, research_focus)
    my_response = ""
    for response in knowledge_response:
        my_response += response.outputs[0].text
    knowledge_data = QA.parse_raw(my_response)
    
    # Debate rounds
    for round_num in range(1, debate_rounds + 1):
        print(f"\n🎭 DEBATE ROUND {round_num} OF {debate_rounds}")
        
        # Evaluate current QA pairs
        print("⚖️ EVALUATING CURRENT KNOWLEDGE EXCHANGES...")
        assessment_response = evaluate_qa(source_manuscript, knowledge_data, round_num)
        my_assessment = ""
        for response in assessment_response:
            my_assessment += response.outputs[0].text
        
        if round_num < debate_rounds:
            # Generate improved QA pairs based on feedback
            print("🔄 GENERATING IMPROVED KNOWLEDGE EXCHANGES...")
            knowledge_response = generate_qa(source_manuscript, research_focus, my_assessment)
            my_response = ""
            for response in knowledge_response:
                my_response += response.outputs[0].text
            knowledge_data = QA.parse_raw(my_response)
    
    # Archive final round exchanges
    exchanges_preserved = archive_enchanted_dialogues(knowledge_data, my_assessment)
    
    # Extract next research direction from evaluation
    assessment_grimoire = json.loads(my_assessment)
    next_research_focus = assessment_grimoire['next_search_query']
    
    return exchanges_preserved, next_research_focus
#-------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------
# MAIN LOOP - Iterates N=2 times through the entire knowledge acquisition and training process
#-------------------------------------------------------------------------------------------
for iteration in range(2):
    #-------------------------------------------------------------------------------------------
    # LLM INITIALIZATION - Set up the language model with specific parameters
    #-------------------------------------------------------------------------------------------
    sampling_params = SamplingParams(temperature=2.0, min_p=0.3)
    llm = LLM(model="/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411", tensor_parallel_size=4, guided_decoding_backend = "lm-format-enforcer")
    current_research_focus = "large language models"
    preserved_exchanges_count = 0

    print("\n🎓 WAND UNIVERSITY KNOWLEDGE ACQUISITION SYSTEM INITIALIZED 🎓")
    print("="*80)

    #-------------------------------------------------------------------------------------------
    # KNOWLEDGE ACQUISITION LOOP - Search and process research papers
    #-------------------------------------------------------------------------------------------
    research_cycle = 0
    while research_cycle < 2:
        try:
            # Search the grand archives
            for i in tqdm(range(0,2), desc="SEARCHING"):
                print(f"\n🔍 SEARCHING ARCHIVES WITH FOCUS: {current_research_focus}")
                ancient_manuscripts = load_arxiv_papers(current_research_focus, max_results=1)
                the_text = ""            
                for manuscript in tqdm(ancient_manuscripts, desc="PROCESSING MANUSCRIPTS"):
                    the_text += manuscript.text
            
                print("\n" + "="*80)
                print(f"📜 ANALYZING MAGICAL MANUSCRIPT:")
                print("="*80 + "\n")
                print(the_text[:500] + "......")  
                print("\n" + "="*80 + "\n")
                
                exchanges_preserved, next_focus = synthesize_and_evaluate_knowledge(the_text, current_research_focus)
                preserved_exchanges_count += exchanges_preserved
                print(f"📊 PRESERVED {exchanges_preserved} MAGICAL EXCHANGES. TOTAL IN ARCHIVES: {preserved_exchanges_count}")
                print(f"🎯 NEXT RESEARCH FOCUS: {next_focus}\n")
                
                current_research_focus = next_focus
                print(f"🔄 COMPLETED RESEARCH CYCLE {research_cycle + 1}. NEXT FOCUS: {current_research_focus}")
                research_cycle += 1
            
        except Exception as e:
            print(f"⚡ MAGICAL ANOMALY DETECTED: {str(e)}")
            continue

    print(f"\n🏆 GRAND ARCHIVE COMPLETE! TOTAL MAGICAL EXCHANGES PRESERVED: {preserved_exchanges_count} 🏆")

    #-------------------------------------------------------------------------------------------
    # CLEANUP - Free GPU memory and shutdown services so that we can do lora training
    #-------------------------------------------------------------------------------------------
    destroy_model_parallel()
    destroy_distributed_environment()
    del llm.llm_engine.model_executor
    del llm
    with contextlib.suppress(AssertionError):
        torch.distributed.destroy_process_group()
    gc.collect()
    torch.cuda.empty_cache()
    ray.shutdown()
    print("Successfully delete the llm pipeline and free the GPU memory.")

    #-------------------------------------------------------------------------------------------
    # LORA TRAINING SETUP - Initialize configuration for fine-tuning
    #-------------------------------------------------------------------------------------------
    model_id = "mistralai/Mistral-7B-v0.1"
    output_dir = "./wand_university_lora"
    num_train_epochs = 1
    per_device_train_batch_size = 1
    learning_rate = 1e-5
    max_seq_length = 4096
    lora_r = 8
    lora_alpha = 64
    lora_dropout = 0.05

    #-------------------------------------------------------------------------------------------
    # WANDB INITIALIZATION - Set up experiment tracking for lora 
    #-------------------------------------------------------------------------------------------
    wandb.init(
        project="wand-university-lora",
        name="arcane-knowledge-transfer-incremental",
        config={
            "epochs": num_train_epochs,
            "batch_size": per_device_train_batch_size,
            "learning_rate": learning_rate,
            "lora_rank": lora_r,
            "lora_alpha": lora_alpha
        }
    )

    #-------------------------------------------------------------------------------------------
    # DATA PREPARATION - Load and process training data
    #-------------------------------------------------------------------------------------------
    print(f"🔮 SUMMONING TRAINING DATA FROM THE ANCIENT GRIMOIRE... 📚 [Model: {model_id}]")
    df = pd.read_csv('wand_university_training_grimoire.csv', sep='|')
    df = df.dropna()

    #-------------------------------------------------------------------------------------------
    # MODEL INITIALIZATION - Load base student model and configure tokenizer, for now mistral7b
    #-------------------------------------------------------------------------------------------
    print(f"⚡ INVOKING THE SACRED TOKENIZER AND MODEL... 🤖 [Sequence Length: {max_seq_length}]")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', attn_implementation="flash_attention_2", torch_dtype=torch.float16)
    model.enable_input_require_grads()

    for param in model.parameters():
        param.requires_grad = True

    #-------------------------------------------------------------------------------------------
    # LORA CONFIGURATION - Set up LoRA parameters and architecture
    #-------------------------------------------------------------------------------------------
    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=[
            "q_proj",
            "k_proj", 
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            "lm_head",
        ],
        bias="none",
        lora_dropout=lora_dropout,
        task_type="CAUSAL_LM",
    )

    #-------------------------------------------------------------------------------------------
    # MODEL LOADING/CREATION - Either load existing LoRA or create new one if doesn't exist
    #-------------------------------------------------------------------------------------------
    if os.path.exists(output_dir):
        print(f"🔄 LOADING EXISTING LORA ENCHANTMENT FROM {output_dir}...")
        model = PeftModel.from_pretrained(model, output_dir)
    else:
        print(f"✨ CREATING NEW LORA ENCHANTMENT WITH RANK {lora_r} AND ALPHA {lora_alpha}...")
        model = get_peft_model(model, config)

    model.print_trainable_parameters()

    #-------------------------------------------------------------------------------------------
    # DATASET PREPARATION - Format data for training
    #-------------------------------------------------------------------------------------------
    print(f"📜 PREPARING THE SACRED TEXTS... ✨ [Dataset Size: {len(df)} entries]")
    formatted_data = [
        {
            "text": f"Question: {inquiry}\nAnswer: {wisdom}"
        }
        for inquiry, wisdom in zip(df['inquiry'], df['wisdom'])
    ]
    dataset = Dataset.from_list(formatted_data)

    #-------------------------------------------------------------------------------------------
    # TRAINING CONFIGURATION - Set up training arguments
    #-------------------------------------------------------------------------------------------
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        learning_rate=learning_rate,
        report_to="wandb",
        overwrite_output_dir=True,
        save_strategy="no"
    )

    #-------------------------------------------------------------------------------------------
    # TRAINER INITIALIZATION AND TRAINING
    #-------------------------------------------------------------------------------------------
    trainer = SFTTrainer(
        model=model,
        args=args,
        train_dataset=dataset,
        dataset_text_field='text',
        max_seq_length=max_seq_length,
    )

    print(f"🌟 COMMENCING THE SACRED LORA TRAINING RITUAL... 🧙‍♂️ [Epochs: {num_train_epochs}, Learning Rate: {learning_rate}]")
    trainer.train()

    #-------------------------------------------------------------------------------------------
    # MODEL SAVING AND CLEANUP
    #-------------------------------------------------------------------------------------------
    print(f"💾 PRESERVING THE ENHANCED MODEL IN THE ARCANE ARCHIVES... 📦 [Output Directory: {output_dir}]")
    trainer.model.save_pretrained(output_dir)

    print(f"📊 CONCLUDING THE WANDB LOGGING RITUAL... 🔮")
    wandb.finish()

    print(f"🎉 THE SACRED LORA TRAINING RITUAL IS COMPLETE! 🎊 [Total Training Steps: {trainer.state.global_step}]")
    
    #-------------------------------------------------------------------------------------------
    # LORA CLEANUP AND SILLY STUFF TO MAKE IT WORK IN VLLM
    #-------------------------------------------------------------------------------------------
    print(f"🧹 CLEANING THE LORA ENCHANTMENT ARTIFACTS... ✨")
    lora_path = '/home/lain/wand_university_lora/adapter_model.safetensors'
    import safetensors.torch
    tensors = safetensors.torch.load_file(lora_path)

    nonlora_keys = []
    for k in list(tensors.keys()):
        if "lora" not in k:
            nonlora_keys.append(k)

    print(f"🔍 FOUND {len(nonlora_keys)} NON-LORA KEYS TO REMOVE...")
    for k in nonlora_keys:
        del tensors[k]

    print(f"💾 SAVING CLEANED LORA ENCHANTMENT... 📦")
    safetensors.torch.save_file(tensors, '/home/lain/wand_university_lora/adapter_model.safetensors')

    #-------------------------------------------------------------------------------------------
    # MODEL EVALUATION (using lm-evaluation-harness and vllm)
    #-------------------------------------------------------------------------------------------
    print(f"🧪 BEGINNING MODEL EVALUATION RITUAL... 📊")
    !python3 /home/lain/lm-evaluation-harness/lm_eval \
            --model vllm \
            --model_args pretrained=mistralai/Mistral-7B-v0.1,dtype=auto,tensor_parallel_size=4,distributed_executor_backend=ray,enable_lora=True,lora_local_path=/home/lain/wand_university_lora \
            --batch_size 4 \
            --tasks gsm8k_cot \
            --num_fewshot 1 \
            --limit 20 \
            --wandb_args project=rebuttal-response \
            --log_samples \
            --output_path /home/lain/lm-eval-output/ \
            --gen_kwargs "min_p=0.1,temperature=1.0,do_sample=True" \
            --device cuda
    print(f"🧪 MODEL EVALUATION RITUAL COMPLETE! 📊")

INFO 12-25 08:11:17 config.py:2167] Downcasting torch.float32 to torch.float16.
INFO 12-25 08:11:23 config.py:478] This model supports multiple tasks: {'classify', 'reward', 'score', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 12-25 08:11:23 config.py:1216] Defaulting to use mp for distributed inference
WARNING 12-25 08:11:23 arg_utils.py:1086] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-25 08:11:23 config.py:1364] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 12-25 08:11:23 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411', speculative_config=None, tokenizer='/home/lain/text-generation-webui/models/mistralai_Mistral-Large-Instruct-2411', skip_t

Loading safetensors checkpoint shards:   0% Completed | 0/51 [00:00<?, ?it/s]


(VllmWorkerProcess pid=85255) INFO 12-25 08:12:03 model_runner.py:1097] Loading model weights took 57.1291 GB
INFO 12-25 08:12:03 model_runner.py:1097] Loading model weights took 57.1291 GB
(VllmWorkerProcess pid=85254) INFO 12-25 08:12:04 model_runner.py:1097] Loading model weights took 57.1291 GB
(VllmWorkerProcess pid=85253) INFO 12-25 08:12:04 model_runner.py:1097] Loading model weights took 57.1291 GB
(VllmWorkerProcess pid=85254) INFO 12-25 08:12:07 worker.py:241] Memory profiling takes 2.76 seconds
(VllmWorkerProcess pid=85254) INFO 12-25 08:12:07 worker.py:241] the current vLLM instance can use total_gpu_memory (79.14GiB) x gpu_memory_utilization (0.90) = 71.22GiB
(VllmWorkerProcess pid=85254) INFO 12-25 08:12:07 worker.py:241] model weights take 57.13GiB; non_torch_memory takes 0.36GiB; PyTorch activation peak memory takes 0.27GiB; the rest of the memory reserved for KV Cache is 13.46GiB.
(VllmWorkerProcess pid=85255) (VllmWorkerProcess pid=85253) INFO 12-25 08:12:07 worker.py

SEARCHING:   0%|          | 0/2 [00:00<?, ?it/s]


🔍 SEARCHING ARCHIVES WITH FOCUS: large language models


...

📊 CONCLUDING THE WANDB LOGGING RITUAL... 🔮


train/epoch,▁
train/global_step,▁
total_flos,1035383673667584.0
train/epoch,1
train/global_step,279
train_loss,1.42211
train_runtime,60.693
train_samples_per_second,4.597
train_steps_per_second,4.597


🎉 THE SACRED LORA TRAINING RITUAL IS COMPLETE! 🎊 [Total Training Steps: 279]
🧹 CLEANING THE LORA ENCHANTMENT ARTIFACTS... ✨
🔍 FOUND 1 NON-LORA KEYS TO REMOVE...
💾 SAVING CLEANED LORA ENCHANTMENT... 📦
🧪 BEGINNING MODEL EVALUATION RITUAL... 📊


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: rainsilves. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /home/lain/wandb/run-20241225_083507-9tsim5e0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ethereal-mountain-97
wandb: ⭐️ View project at https://wandb.ai/rainsilves/rebuttal-response
wandb: 🚀 View run at https://wandb.ai/rainsilves/rebuttal-response/runs/9tsim5e0
2024-12-25:08:35:08,032 INFO     [__main__.py:279] Verbosity set to INFO
2024-12-25:08:35:08,132 INFO     [__init__.py:491] `group` and `group_alias` keys in TaskConfigs are deprecated and will be removed in v0.4.5 of lm_eval. The new `tag` field will be used to allow for a shortcut to a group of tasks one does not wish to aggregate metrics across. `group`s which aggregate across subtasks must be only defined in a separate group config file, which will be the official way to create groups that support cross-task aggregation as in `mmlu`.

In [9]:
class QA(BaseModel):
    question1: str = Field(description="First general knowledge question about the core topic and concepts")
    answer1: str = Field(description="Clear, factual answer focused on general knowledge")
    question2: str = Field(description="Second general knowledge question about the core topic and concepts") 
    answer2: str = Field(description="Clear, factual answer focused on general knowledge")
    question3: str = Field(description="Third general knowledge question about the core topic and concepts")
    answer3: str = Field(description="Clear, factual answer focused on general knowledge")

DEFAULT_SYSTEM_CONTENT = """You are an expert at creating high-quality training data for language models.
Given academic research content, extract the key concepts and convert them into general knowledge questions and answers.
Questions should:
- Be general and broadly applicable
- Focus on core concepts and ideas
- Be written as standalone questions without referencing any specific research
- Test understanding of the topic area

Answers should:
- Provide clear, factual information
- Be written as standalone knowledge
- Focus on general concepts rather than specific research
- Be useful for general learning about the topic"""

def generate_qa(document_text: str, system_content: str = DEFAULT_SYSTEM_CONTENT, feedback: str = None):
    json_schema = QA.model_json_schema()
    guided_decoding_params = GuidedDecodingParams(json=json_schema)
    sampling_params = SamplingParams(
        guided_decoding=guided_decoding_params,
        temperature=1.5,
        max_tokens=4000,
        presence_penalty=0.1,
        frequency_penalty=0.1,
        min_p=0.4,
        stop=["<|eot_id|>"]
    )
    
    messages = [
        {"role": "system", "content": system_content}
    ]
    
    if feedback:
        messages.append({"role": "user", "content": f"Based on this feedback, please generate improved questions and answers:\n\n{feedback}\n\nOriginal text: {document_text}"})
    else:
        messages.append({"role": "user", "content": f"Based on these concepts, generate 3 general knowledge questions and answers about: {document_text}"})
    
    return llm.chat(
        messages=messages,
        sampling_params=sampling_params
    )

class QAEvaluation(BaseModel):
    qa_pair1_arguments_for: list[str] = Field(description="Arguments in favor of including first QA pair")
    qa_pair1_arguments_against: list[str] = Field(description="Arguments against including first QA pair") 
    qa_pair1_include: Literal["y", "n"] = Field(description="Whether to include first QA pair")

    qa_pair2_arguments_for: list[str] = Field(description="Arguments in favor of including second QA pair")
    qa_pair2_arguments_against: list[str] = Field(description="Arguments against including second QA pair")
    qa_pair2_include: Literal["y", "n"] = Field(description="Whether to include second QA pair")

    qa_pair3_arguments_for: list[str] = Field(description="Arguments in favor of including third QA pair")
    qa_pair3_arguments_against: list[str] = Field(description="Arguments against including third QA pair")
    qa_pair3_include: Literal["y", "n"] = Field(description="Whether to include third QA pair")
    
    next_search_query: str = Field(description="Recommended next search query for finding related but diverse papers")

def evaluate_qa(document_text: str, qa_output: QA, round_num: int):
    json_schema = QAEvaluation.model_json_schema()
    guided_decoding_params = GuidedDecodingParams(json=json_schema)
    sampling_params = SamplingParams(
        guided_decoding=guided_decoding_params,
        temperature=1.5,
        max_tokens=4000,
        presence_penalty=0.1,
        frequency_penalty=0.1,
        min_p=0.4,
        stop=["<|eot_id|>"]
    )
    
    eval_system_prompt = f"""You are an expert evaluator of training data quality for language models.
    This is debate round {round_num}. Be increasingly critical with each round.
    For each question-answer pair:
    1. List key arguments for and against including it (considering accuracy, clarity, generalizability)
    2. Make a binary decision (y/n) if it should be included in the final dataset
    
    Additionally, recommend a search query for finding the next paper to analyze. The query should:
    - Be related to but very distinct from the current paper's topic
    - Help build a diverse dataset while maintaining topical coherence
    - Potentially focus on an interesting direction suggested by the current paper"""
    
    eval_user_prompt = f"""Evaluate these QA pairs generated from the paper:
    
    Original Paper Text: {document_text}
    
    QA Pair 1:
    Q: {qa_output.question1}
    A: {qa_output.answer1}
    
    QA Pair 2:
    Q: {qa_output.question2}
    A: {qa_output.answer2}
    
    QA Pair 3:
    Q: {qa_output.question3}
    A: {qa_output.answer3}
    
    Provide a structured evaluation following the schema, including a recommended next search query."""

    messages = [
        {"role": "system", "content": eval_system_prompt},
        {"role": "user", "content": eval_user_prompt}
    ]
    
    return llm.chat(
        messages=messages,
        sampling_params=sampling_params
    )

def archive_enchanted_dialogues(knowledge_exchange_data, assessment_data, archive_path="wand_university_training_grimoire.csv"):
    """Archive validated magical dialogues and their scholarly assessments for the Wand University curriculum"""
    print("\n🌟 INITIATING ARCHIVAL PROCESS OF ENCHANTED DIALOGUES 🌟")
    
    # Transform assessment data from arcane notation to comprehensible format
    assessment_grimoire = json.loads(assessment_data)
    
    # Initialize collection of validated knowledge exchanges
    validated_exchanges = []
    
    # Evaluate each knowledge exchange through our rigorous magical standards
    for exchange_id in tqdm(range(1,4), desc="VALIDATING KNOWLEDGE EXCHANGES"):
        if assessment_grimoire[f'qa_pair{exchange_id}_include'] == 'y':
            # Synthesize supporting and challenging arguments
            supporting_thesis = assessment_grimoire.get(f'qa_pair{exchange_id}_arguments_for', [])
            counterpoints = assessment_grimoire.get(f'qa_pair{exchange_id}_arguments_against', [])
            
            print(f"\n✨ ACCEPTED KNOWLEDGE EXCHANGE {exchange_id}:")
            print(f"📝 INQUIRY: {getattr(knowledge_exchange_data, f'question{exchange_id}')}")
            print(f"💭 RESPONSE: {getattr(knowledge_exchange_data, f'answer{exchange_id}')}")
            print("✅ POSITIVE FEEDBACK:")
            for strength in supporting_thesis:
                print(f"  • {strength}")
            
            validated_exchanges.append({
                'inquiry': getattr(knowledge_exchange_data, f'question{exchange_id}'),
                'wisdom': getattr(knowledge_exchange_data, f'answer{exchange_id}'),
                'supporting_thesis': ', '.join(supporting_thesis),
                'counterpoints': ', '.join(counterpoints),
                'search_query': assessment_grimoire['next_search_query']
            })
        else:
            print(f"\n⚠️ DISCARDED KNOWLEDGE EXCHANGE {exchange_id}:")
            print(f"📝 INQUIRY: {getattr(knowledge_exchange_data, f'question{exchange_id}')}")
            print(f"💭 RESPONSE: {getattr(knowledge_exchange_data, f'answer{exchange_id}')}")
            print("❌ CRITICAL FEEDBACK:")
            for critique in assessment_grimoire.get(f'qa_pair{exchange_id}_arguments_against', []):
                print(f"  • {critique}")
    
    print("\n📚 TRANSFORMING KNOWLEDGE INTO STRUCTURED FORMAT...")
    # Transform into structured knowledge format
    knowledge_codex = pd.DataFrame(validated_exchanges)
    
    # Preserve in the grand archives
    print("💾 PRESERVING IN THE GRAND ARCHIVES...")
    if Path(archive_path).exists():
        knowledge_codex.to_csv(archive_path, mode='a', header=False, index=False, sep='|')
    else:
        knowledge_codex.to_csv(archive_path, index=False, sep='|')
    
    print(f"✨ SUCCESSFULLY ARCHIVED {len(validated_exchanges)} EXCHANGES ✨")
    return len(validated_exchanges)

def synthesize_and_evaluate_knowledge(source_manuscript, research_focus, debate_rounds=3):
    """Synthesize and evaluate magical knowledge exchanges through our proprietary arcane processes"""
    print("\n🔮 INITIATING KNOWLEDGE SYNTHESIS AND EVALUATION 🔮")
    
    # Initial generation
    print("📖 GENERATING INITIAL KNOWLEDGE EXCHANGES...")
    knowledge_response = generate_qa(source_manuscript, research_focus)
    my_response = ""
    for response in knowledge_response:
        my_response += response.outputs[0].text
    knowledge_data = QA.parse_raw(my_response)
    
    # Debate rounds
    for round_num in range(1, debate_rounds + 1):
        print(f"\n🎭 DEBATE ROUND {round_num} OF {debate_rounds}")
        
        # Evaluate current QA pairs
        print("⚖️ EVALUATING CURRENT KNOWLEDGE EXCHANGES...")
        assessment_response = evaluate_qa(source_manuscript, knowledge_data, round_num)
        my_assessment = ""
        for response in assessment_response:
            my_assessment += response.outputs[0].text
        
        if round_num < debate_rounds:
            # Generate improved QA pairs based on feedback
            print("🔄 GENERATING IMPROVED KNOWLEDGE EXCHANGES...")
            knowledge_response = generate_qa(source_manuscript, research_focus, my_assessment)
            my_response = ""
            for response in knowledge_response:
                my_response += response.outputs[0].text
            knowledge_data = QA.parse_raw(my_response)
    
    # Archive final round exchanges
    exchanges_preserved = archive_enchanted_dialogues(knowledge_data, my_assessment)
    
    # Extract next research direction from evaluation
    assessment_grimoire = json.loads(my_assessment)
    next_research_focus = assessment_grimoire['next_search_query']
    
    return exchanges_preserved, next_research_focus

🔮 SUMMONING TRAINING DATA FROM THE ANCIENT GRIMOIRE... 📚 [Model: mistralai/Mistral-7B-v0.1]
⚡ INVOKING THE SACRED TOKENIZER AND MODEL... 🤖 [Sequence Length: 4096]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔄 LOADING EXISTING LORA ENCHANTMENT FROM ./wand_university_lora...
trainable params: 0 || all params: 7,262,992,384 || trainable%: 0.0000
📜 PREPARING THE SACRED TEXTS... ✨ [Dataset Size: 243 entries]


/home/lain/micromamba/envs/vllm/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/lain/micromamba/envs/vllm/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/lain/micromamba/envs/vllm/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/243 [00:00<?, ? examples/s]

...

Step,Training Loss


💾 PRESERVING THE ENHANCED MODEL IN THE ARCANE ARCHIVES... 📦 [Output Directory: ./wand_university_lora]


/home/lain/micromamba/envs/vllm/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


train/epoch,▁
train/global_step,▁
total_flos,934951970820096.0
train/epoch,1
train/global_step,243
train_loss,1.40551
train_runtime,49.5257
train_samples_per_second,4.907
train_steps_per_second,4.907


🎉 THE SACRED LORA TRAINING RITUAL IS COMPLETE! 🎊 [Total Training Steps: 243]
